In [105]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Imports

In [106]:
import os
import pandas as pd

## Config

In [107]:
loc_data = os.getenv("LOC_DATA")
loc_train = f"{loc_data}/train.csv"
loc_test = f"{loc_data}/test.csv"
loc_stores = f"{loc_data}/stores.csv"
loc_oil = f"{loc_data}/oil.csv"
loc_transactions = f"{loc_data}/transactions.csv"
loc_holiday_events = f"{loc_data}/holidays_events.csv"

loc_pre = os.getenv("LOC_PRE")
loc_pre_train = f"{loc_pre}/train.csv"
loc_pre_test = f"{loc_pre}/test.csv"

## Loading data

In [108]:
df_train = pd.read_csv(loc_train,index_col=[0],parse_dates=["date"])
df_train.head(3)

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0


In [109]:
df_test = pd.read_csv(loc_test,index_col=[0],parse_dates=["date"]).reset_index()
df_test.head(3)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2


In [110]:
df_holidays_events = pd.read_csv(loc_holiday_events,parse_dates=["date"])
df_holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [111]:
df_oil = pd.read_csv(loc_oil,parse_dates=["date"])
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [112]:
df_stores = pd.read_csv(loc_stores)
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [113]:
df_transactions = pd.read_csv(loc_transactions,parse_dates=["date"])
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Placing target on position 0

I have tried sagemaker and it required target to be on pos 0

In [114]:
df_train.insert(0,"sales",df_train.pop("sales"))
df_train

,sales,date,store_nbr,family,onpromotion
id,,,,,
0,0.000,2013-01-01,1,AUTOMOTIVE,0
1,0.000,2013-01-01,1,BABY CARE,0
2,0.000,2013-01-01,1,BEAUTY,0
3,0.000,2013-01-01,1,BEVERAGES,0
4,0.000,2013-01-01,1,BOOKS,0
...,...,...,...,...,...
3000883,438.133,2017-08-15,9,POULTRY,0
3000884,154.553,2017-08-15,9,PREPARED FOODS,1
3000885,2419.729,2017-08-15,9,PRODUCE,148


### Include Holiday events naively

In [115]:
def joinHolidayEvents(df: pd.DataFrame):
    return df.merge(df_holidays_events,how="left",on="date")
    # return pd.merge(df,df_holidays_events,how="left",on="date")


In [116]:
df_train = joinHolidayEvents(df_train)
df_test = joinHolidayEvents(df_test)

### Include Oil data naively

In [117]:
def joinOilData(df: pd.DataFrame):
    return pd.merge(df,df_oil,how="left",on="date")

In [118]:
df_train = joinOilData(df_train)
df_test = joinOilData(df_test)

### Include Stores data naively

In [119]:
def joinStoresData(df: pd.DataFrame):
    return pd.merge(df,df_stores,how="left",on="store_nbr")

In [120]:
df_train = joinStoresData(df_train)
df_test = joinStoresData(df_test)

### Include Transactions naively

In [121]:
def joinTransactionsData(df: pd.DataFrame):
    return pd.merge(df,df_transactions,how="left",on=["store_nbr","date"])

In [122]:
df_train = joinTransactionsData(df_train)
df_test = joinTransactionsData(df_test)

### Parse dates into time step variables

In [123]:
# Parses year,month,day into its own columns in place for the given df
def parseDate(df: pd.DataFrame):
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    df.drop("date",axis=1,inplace=True)

In [124]:
parseDate(df_train)
parseDate(df_test)

### Export preprocessed data

In [126]:
df_train.to_csv(loc_pre_train,index=False)
df_test.to_csv(loc_pre_test,index=False)